In [1]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import collections
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
# Enable eager execution (for TFF compatibility)
tf.compat.v1.enable_v2_behavior()

2025-02-04 11:05:08.732446: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-04 11:05:08.886488: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 11:05:08.886537: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 11:05:08.887297: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 11:05:08.948664: I tensorflow/core/platform/cpu_feature_g

In [8]:
header_columns = [
    "Flow ID", "Src IP", "Src Port", "Dst IP", "Dst Port", "Protocol", "Timestamp",
    "Flow Duration", "Total Fwd Packet", "Total Bwd packets", "Total Length of Fwd Packet",
    "Total Length of Bwd Packet", "Fwd Packet Length Max", "Fwd Packet Length Min",
    "Fwd Packet Length Mean", "Fwd Packet Length Std", "Bwd Packet Length Max",
    "Bwd Packet Length Min", "Bwd Packet Length Mean", "Bwd Packet Length Std",
    "Flow Bytes/s", "Flow Packets/s", "Flow IAT Mean", "Flow IAT Std", "Flow IAT Max",
    "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Mean", "Fwd IAT Std", "Fwd IAT Max",
    "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean", "Bwd IAT Std", "Bwd IAT Max",
    "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags",
    "Fwd Header Length", "Bwd Header Length", "Fwd Packets/s", "Bwd Packets/s",
    "Packet Length Min", "Packet Length Max", "Packet Length Mean", "Packet Length Std",
    "Packet Length Variance", "FIN Flag Count", "SYN Flag Count", "RST Flag Count",
    "PSH Flag Count", "ACK Flag Count", "URG Flag Count", "CWR Flag Count", "ECE Flag Count",
    "Down/Up Ratio", "Average Packet Size", "Fwd Segment Size Avg", "Bwd Segment Size Avg",
    "Fwd Bytes/Bulk Avg", "Fwd Packet/Bulk Avg", "Fwd Bulk Rate Avg", "Bwd Bytes/Bulk Avg",
    "Bwd Packet/Bulk Avg", "Bwd Bulk Rate Avg", "Subflow Fwd Packets", "Subflow Fwd Bytes",
    "Subflow Bwd Packets", "Subflow Bwd Bytes", "FWD Init Win Bytes", "Bwd Init Win Bytes",
    "Fwd Act Data Pkts", "Fwd Seg Size Min", "Active Mean", "Active Std", "Active Max",
    "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min", "Activity", "Stage"
]


In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import os
from glob import glob


def load_multiple_files(data_dir):
    """Load and combine multiple CSV files from a directory using only desired columns."""
    file_pattern = os.path.join(data_dir, "*.csv")
    file_list = glob(file_pattern)
    
    if not file_list:
        raise ValueError(f"No CSV files found in {data_dir}")
    
    dfs = []
    for file in file_list:
        # Read only the first 85 columns (or use the list of indices you prefer)
        df = pd.read_csv(file, usecols=list(range(85)))
        print(f"Loading {file}: {df.shape} rows")
        dfs.append(df)
    
    combined_df = pd.concat(dfs, axis=0, ignore_index=True)
    # Select only the valid columns from our header_columns list.
    combined_df = combined_df[header_columns]

    print(f"\nLoaded {len(file_list)} files")
    print(f"Combined dataset shape: {combined_df.shape}")
    print(f"Total samples: {len(combined_df)}")
    print(f"Features: {combined_df.columns.tolist()}")
    print("\nStage distribution:")
    print(combined_df.groupby('Stage').size())
    
    return combined_df

def preprocess_data(df):
    """Preprocessing pipeline using only the Stage feature as the target label."""
    print("\nPreprocessing Steps:")
    print(f"Initial shape: {df.shape}")
    
    # If needed, store Flow ID separately and drop it
    if 'Flow ID' in df.columns:
        flow_ids = df['Flow ID'].values
        df = df.drop('Flow ID', axis=1)
        print("Dropped Flow ID column")
    
    # Handle missing values
    print(f"Missing values before handling: {df.isna().sum().sum()}")
    # Fill numeric columns with median
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
    # Fill categorical columns with mode
    categorical_cols = df.select_dtypes(exclude=[np.number]).columns
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    print(f"Shape after handling missing values: {df.shape}")
    
    # Remove duplicates
    df = df.drop_duplicates()
    print(f"Shape after removing duplicates: {df.shape}")
    
    # Drop the Activity column because we only use Stage for classification.
    if 'Activity' in df.columns:
        df = df.drop(columns=['Activity'])
        print("Dropped Activity column; only Stage will be used as label.")
    
    # Separate features and label
    # Exclude 'Stage' from features since it is the target label.
    feature_cols = [col for col in df.columns if col != 'Stage']
    
    # Convert non-numeric features (if any) to numeric.
    for col in feature_cols:
        if df[col].dtype == 'object':
            print(f"Converting {col} to numeric...")
            # Example: if it's an IP address, convert it to an integer representation.
            if 'IP' in col:
                df[col] = df[col].apply(lambda x: int(''.join([i.zfill(3) for i in str(x).split('.')])) )
            else:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
    
    X = df[feature_cols].values.astype(np.float32)
    stage = df['Stage'].values
    
    # Encode the Stage labels only
    stage_encoder = LabelEncoder()
    y_stage = stage_encoder.fit_transform(stage)
    
    print("\nLabel Encoding Information:")
    print("Stage classes:", stage_encoder.classes_)
    
    # Split dataset using only the stage label for stratification.
    X_train, X_test, y_stage_train, y_stage_test = train_test_split(
        X, y_stage,
        test_size=0.3,
        stratify=y_stage,
        random_state=42
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    print(f"\nFinal shapes:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    
    return X_train, X_test, y_stage_train, y_stage_test, scaler, stage_encoder

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import os
from glob import glob

def create_federated_clients(X_train, y_stage_train, num_clients=5):
    """Create federated clients using only stage labels."""
    # Split data into horizontal partitions for federated learning
    client_data = np.array_split(X_train, num_clients)
    client_stage_labels = np.array_split(y_stage_train, num_clients)
    
    for i, (data, stage_labels) in enumerate(zip(client_data, client_stage_labels)):
        print(f"\nClient {i+1}:")
        print(f"Samples: {len(data)}")
        print(f"Unique stages: {len(np.unique(stage_labels))}")
    
    return client_data, client_stage_labels

def main_data_pipeline(data_dir, num_clients=5):
    """Main preprocessing workflow"""
    combined_df = load_multiple_files(data_dir)
    
    X_train, X_test, y_stage_train, y_stage_test, scaler, stage_encoder = preprocess_data(combined_df)
    
    # Create federated clients with only stage labels
    client_data, client_stage_labels = create_federated_clients(X_train, y_stage_train, num_clients)
    
    return client_data, client_stage_labels, X_test, y_stage_test, scaler, stage_encoder

if __name__ == "__main__":
    data_directory = "/home/brahim/Desktop/PFE2025/Dataset/csv/"
    
    # Run preprocessing pipeline
    client_data, client_stage_labels, X_test, y_stage_test, scaler, stage_encoder = main_data_pipeline(data_directory)
    
    print("\nPreprocessing complete!")
    print(f"Training data shape: {sum([c.shape[0] for c in client_data])} samples")
    print(f"Test data shape: {X_test.shape[0]} samples")
    print(f"Stage classes: {stage_encoder.classes_}")


Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-monday-pvt.pcap_Flow.csv: (3404, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-public-wednesday.pcap_Flow.csv: (17487, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-public-thursday.pcap_Flow.csv: (9685, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-pvt-wednesday.pcap_Flow.csv: (1437, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-pvt-thursday.pcap_Flow.csv: (4113, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-tcpdump-friday.pcap_Flow.csv: (7361, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-monday.pcap_Flow.csv: (8728, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-tcpdump-pvt-friday.pcap_Flow.csv: (2618, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-pvt-tuesday.pcap_Flow.csv: (2615, 85) rows
Loading /home/brahim/Desktop/PFE2025/Dataset/csv/enp0s3-public-tuesday.pcap_Flow.csv: (29242

In [35]:
def create_ids_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


num_selected_features = client_data[0].shape[1]  # the number of features you decide to use

def model_fn():
    keras_model = create_ids_model(input_shape=(num_selected_features,))
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=client_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Build the federated averaging process:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(model_fn=model_fn,client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.001))

state = iterative_process.initialize()

num_rounds = 20
for round_num in range(1, num_rounds + 1):
    state, metrics = iterative_process.next(state, client_data)
    print(f'Round {round_num:2d}, Metrics={metrics}')

# Evaluate the final global model:
global_model = create_ids_model(input_shape=(num_selected_features,))
state.model.assign_weights_to(global_model)

global_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)
test_metrics = global_model.evaluate(X_test, y_stage_test, verbose=0)
print(f"\nTest metrics: {test_metrics}")


AttributeError: 'numpy.ndarray' object has no attribute 'element_spec'

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import collections

class ClientKNN:
    """KNN implementation for individual clients"""
    def __init__(self, k=3):
        self.k = k
        self.X = None
        self.y = None
        
    def store_data(self, X, y):
        """Store local data"""
        self.X = X
        self.y = y
    
    def predict(self, X_test):
        """Make predictions using only local data"""
        predictions = []
        for x in X_test:
            distances = np.sqrt(np.sum((self.X - x) ** 2, axis=1))
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y[k_indices]
            prediction = collections.Counter(k_nearest_labels).most_common(1)[0][0]
            predictions.append(prediction)
        return np.array(predictions)

class FederatedServer:
    """Server that coordinates predictions without storing data"""
    def __init__(self):
        self.client_ids = set()
    
    def register_client(self, client_id):
        """Register a new client"""
        self.client_ids.add(client_id)
    
    def aggregate_predictions(self, client_predictions):
        """Aggregate predictions from all clients using majority voting"""
        all_predictions = np.array(list(client_predictions.values()))
        final_predictions = []
        
        for i in range(all_predictions.shape[1]):
            votes = collections.Counter(all_predictions[:, i])
            final_predictions.append(votes.most_common(1)[0][0])
            
        return np.array(final_predictions)

def simulate_federated_prediction(X_test, clients):
    """Simulate the federated prediction process"""
    server = FederatedServer()
    client_predictions = {}
    
    # Each client makes predictions locally
    for client_id, client in clients.items():
        server.register_client(client_id)
        predictions = client.predict(X_test)
        client_predictions[client_id] = predictions
    
    # Server aggregates results without seeing the actual data
    final_predictions = server.aggregate_predictions(client_predictions)
    return final_predictions

# Example usage:
"""
# Initialize clients with their local data
clients = {}
for client_id in range(3):  # Example with 3 clients
    clients[client_id] = ClientKNN(k=3)
    # Each client stores only their local data
    clients[client_id].store_data(X_local[client_id], y_local[client_id])

# Make federated predictions
predictions = simulate_federated_prediction(X_test, clients)

# Calculate metrics (can be done either at server or client side)
metrics = {
    'accuracy': accuracy_score(y_test, predictions),
    'precision': precision_score(y_test, predictions),
    'recall': recall_score(y_test, predictions),
    'f1_score': f1_score(y_test, predictions)
}
"""

"\n# Initialize clients with their local data\nclients = {}\nfor client_id in range(3):  # Example with 3 clients\n    clients[client_id] = ClientKNN(k=3)\n    # Each client stores only their local data\n    clients[client_id].store_data(X_local[client_id], y_local[client_id])\n\n# Make federated predictions\npredictions = simulate_federated_prediction(X_test, clients)\n\n# Calculate metrics (can be done either at server or client side)\nmetrics = {\n    'accuracy': accuracy_score(y_test, predictions),\n    'precision': precision_score(y_test, predictions),\n    'recall': recall_score(y_test, predictions),\n    'f1_score': f1_score(y_test, predictions)\n}\n"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def balance_network_datasets(normal_file, apt_file, output_file1, output_file2):
    """
    Combine and balance network traffic datasets
    
    Parameters:
    normal_file: path to file containing normal traffic
    apt_file: path to file containing APT traffic
    output_file1: path for first balanced output file
    output_file2: path for second balanced output file
    """
    
    # Read the datasets
    print("Reading input files...")
    normal_df = pd.read_csv(normal_file)
    apt_df = pd.read_csv(apt_file)
    
    # Add labels if they don't exist
    normal_df['label'] = 0  # 0 for normal traffic
    apt_df['label'] = 1     # 1 for APT traffic
    
    # Calculate sizes for balanced datasets
    min_normal = len(normal_df)
    min_apt = len(apt_df)
    
    # Determine samples needed per class for 50-50 split
    samples_per_class = min(min_normal, min_apt)
    
    print(f"Original dataset sizes:")
    print(f"Normal traffic: {min_normal} samples")
    print(f"APT traffic: {min_apt} samples")
    print(f"Will use {samples_per_class} samples per class for balance")
    
    # Randomly sample equal amounts from each dataset
    if min_normal > samples_per_class:
        normal_df = normal_df.sample(n=samples_per_class, random_state=42)
    if min_apt > samples_per_class:
        apt_df = apt_df.sample(n=samples_per_class, random_state=42)
    
    # Combine the datasets
    combined_df = pd.concat([normal_df, apt_df], ignore_index=True)
    
    # Shuffle the combined dataset
    combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Split into two equal parts
    df1, df2 = train_test_split(combined_df, test_size=0.5, stratify=combined_df['label'], random_state=42)
    
    # Verify balance in each dataset
    print("\nDataset 1 distribution:")
    print(df1['label'].value_counts(normalize=True))
    print("\nDataset 2 distribution:")
    print(df2['label'].value_counts(normalize=True))
    
    # Save to files
    df1.to_csv(output_file1, index=False)
    df2.to_csv(output_file2, index=False)
    !
    print(f"\nSaved balanced datasets:")
    print(f"Dataset 1: {len(df1)} samples saved to {output_file1}")
    print(f"Dataset 2: {len(df2)} samples saved to {output_file2}")
    
    return df1, df2

def verify_balance(df1, df2):
    """Verify that the datasets are properly balanced"""
    
    print("\nBalance Verification:")
    print("\nDataset 1:")
    print(f"Total samples: {len(df1)}")
    print("Class distribution:")
    print(df1['label'].value_counts(normalize=True))
    
    print("\nDataset 2:")
    print(f"Total samples: {len(df2)}")
    print("Class distribution:")
    print(df2['label'].value_counts(normalize=True))

if __name__ == "__main__":
    # File paths - replace with your actual file paths
    normal_traffic_file = "normal_traffic.csv"
    apt_traffic_file = "apt_traffic.csv"
    output_dataset1 = "balanced_dataset1.csv"
    output_dataset2 = "balanced_dataset2.csv"
    
    # Process the datasets
    df1, df2 = balance_network_datasets(
        normal_traffic_file,
        apt_traffic_file,
        output_dataset1,
        output_dataset2
    )
    
    # Verify the balance
    verify_balance(df1, df2)